In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gastroesophageal_reflux_disease_(GERD)"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Gastroesophageal reflux disease (GERD)
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Define key terms relevant to GERD
key_terms = ["esophageal", "stomach", "gastro", "reflux", "gastric", "esophagus"]

# Start with no match, then find the best match based on similarity to target trait
best_match = None
best_match_score = 0
min_threshold = 1  # Require at least 1 matching term

for subdir in subdirectories:
    if not os.path.isdir(os.path.join(tcga_root_dir, subdir)) or subdir.startswith('.'):
        continue
        
    subdir_lower = subdir.lower()
    
    # Check for exact matches
    if target_trait in subdir_lower:
        best_match = subdir
        print(f"Found exact match: {subdir}")
        break
    
    # Calculate a score based on key terms
    score = 0
    for term in key_terms:
        if term in subdir_lower:
            score += 1
    
    # Check for partial matches with threshold
    if score > best_match_score and score >= min_threshold:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Use the best match if found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}. This trait may not be directly represented in the TCGA dataset.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Stomach_Cancer_(STAD) (score: 1)
Selected directory: TCGA_Stomach_Cancer_(STAD)
Clinical file: TCGA.STAD.sampleMap_STAD_clinicalMatrix
Genetic file: TCGA.STAD.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'antireflux_treatment', 'antireflux_treatment_type', 'barretts_esophagus', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'city_of_procurement', 'country_of_procurement', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'family_history_of_stomach_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'h_pylori_infection', 'histological_

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify columns related to age
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']

# Identify columns related to gender
candidate_gender_cols = ['gender']

# Load the clinical data if not already loaded
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Stomach_Cancer_(STAD)"))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age-related columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Preview gender-related columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

# Display the previews
print("Age column previews:")
print(age_preview)
print("\nGender column previews:")
print(gender_preview)


Age column previews:
{'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 'days_to_birth': [nan, nan, -18698.0, -22792.0, -19014.0]}

Gender column previews:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
import numpy as np

# Selecting the most appropriate columns for age and gender
age_cols_data = {'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 
                'days_to_birth': [np.nan, np.nan, -18698.0, -22792.0, -19014.0]}

gender_cols_data = {'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

# Selecting age column
# The 'age_at_initial_pathologic_diagnosis' column has more non-null values and 
# represents age directly rather than requiring conversion
age_col = 'age_at_initial_pathologic_diagnosis' if 'age_at_initial_pathologic_diagnosis' in age_cols_data else None

# Selecting gender column
# The 'gender' column contains standard gender values
gender_col = 'gender' if 'gender' in gender_cols_data else None

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 450)
Clinical data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/TCGA.csv
Clinical data shape: (580, 3)
Number of samples in clinical data: 580
Number of samples in genetic data: 450
Number of common samples: 450
Linked data shape: (450, 19851)


Data shape after handling missing values: (450, 19851)
For the feature 'Gastroesophageal_reflux_disease_(GERD)', the least common label is '0' with 35 occurrences. This represents 7.78% of the dataset.
The distribution of the feature 'Gastroesophageal_reflux_disease_(GERD)' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 67.0
  75%: 73.0
Min: 30.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 159 occurrences. This represents 35.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/TCGA.csv
Preprocessing completed.
